In [1]:
import pandas as pd
import csv
#import boto3
#import xgboost as xgb
import numpy as np

In [2]:
#LOS_Base =  pd.read_csv("/home/siddarth/LOS_data/LOS_Base_V3.csv")
LOS_Base =  pd.read_excel("D:/Varthana/PIPELINE (1.xlsx")
LOS_Base.shape

(11311, 47)

#### Convert dates to date format

In [3]:
LOS_Base["LOGIN_DATE"] = pd.to_datetime(LOS_Base["LOGIN_DATE"]) 
LOS_Base["BUREAU_REVIEW_START_DATE"] = pd.to_datetime(LOS_Base["BUREAU_REVIEW_START_DATE"]) 
LOS_Base["REJECTION_DATE"] = pd.to_datetime(LOS_Base["REJECTION_DATE"]) 
LOS_Base["PD_DATE"] = pd.to_datetime(LOS_Base["PD_DATE"]) 
LOS_Base["RECOMMENDED_DATE"] = pd.to_datetime(LOS_Base["RECOMMENDED_DATE"]) 
LOS_Base["SOFT_SANCTION_APPROVAL_DATE"] = pd.to_datetime(LOS_Base["SOFT_SANCTION_APPROVAL_DATE"]) 
LOS_Base["FINAL_SANCTION_DATE"] = pd.to_datetime(LOS_Base["FINAL_SANCTION_DATE"]) 
LOS_Base["DISBURSEMENT_DATE"] = pd.to_datetime(LOS_Base["DISBURSEMENT_DATE"]) 
LOS_Base["DOC_COLL_DATE"] = pd.to_datetime(LOS_Base["DOC_COLL_DATE"]) 
LOS_Base["APP_UW_COMPLETE_DATE"] = pd.to_datetime(LOS_Base["APP_UW_COMPLETE_DATE"]) 
LOS_Base["TECHNICAL_START_DATE"] = pd.to_datetime(LOS_Base["TECHNICAL_START_DATE"]) 
LOS_Base["START_DATE"] = pd.to_datetime(LOS_Base["START_DATE"]) 

#### Retaining records with earliest disbursement dates

In [4]:
LOS_Base = LOS_Base.sort_values("DISBURSEMENT_DATE",ascending=True )

In [5]:
LOS_Base_1 = LOS_Base.drop_duplicates('SZ_APPLICATION_NO').sort_index()

In [6]:
###### adding prefix 
LOS_Base_1 = LOS_Base_1.add_prefix("LOS_Base_")

### 1. Loan details addition

In [7]:
Loan_details =  pd.read_csv("D:/Varthana/LOS_LOAN_DETAILS_07-Sep-20.csv")

In [8]:
Loan_details = Loan_details.add_prefix("Loan_details_")

In [9]:
df_M_1 = pd.merge(LOS_Base_1, Loan_details,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="Loan_details_APPLICATION_NO", indicator=True)

In [10]:
df_M_1["_merge"].value_counts()/df_M_1["_merge"].count()

both          0.998267
left_only     0.001733
right_only    0.000000
Name: _merge, dtype: float64

### School details addition

In [11]:
schl_det = pd.read_csv("D:/Varthana/school_details_updated.csv", low_memory =  False)

In [12]:
schl_det = schl_det.add_prefix("schl_det_")

In [13]:
df_M_1 = df_M_1.drop("_merge", axis=1)

In [14]:
df_M_2 = pd.merge(df_M_1, schl_det,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="schl_det_SZ_APPLICATION_NO", indicator=True)

In [15]:
df_M_2["_merge"].value_counts()/df_M_2["_merge"].count()

both          0.634647
left_only     0.365353
right_only    0.000000
Name: _merge, dtype: float64

### Obligations data addition


In [16]:
oblig_base =  pd.read_csv("D:/Varthana/OBLIGATION_DETAILS_07-Sep-20.csv")

In [17]:
oblig_base["Open_loan_amt"] = np.where(oblig_base["LOAN_STATUS"] == "OPEN", oblig_base["LOAN_AMOUNT"],0)

In [18]:
oblig_base["Open_emi_amt"] = np.where(oblig_base["LOAN_STATUS"] == "OPEN", oblig_base["EMI_AMOUNT"],0)

In [19]:
oblig_base["pos_closed"] = np.where(oblig_base["LOAN_STATUS"] == "CLOSE", oblig_base["CURRENT_POS"],0)

In [20]:
oblig_base["derogs"] = np.where(oblig_base["C_DEROGFLAG"] == "Y", 1,0)

In [21]:
oblig_aggreg = pd.pivot_table(oblig_base, values=['LOAN_AMOUNT','EMI_AMOUNT','CURRENT_POS','Open_loan_amt','Open_emi_amt','pos_closed','derogs'], index=['SZ_APPLICATION_NO'],aggfunc=np.sum)

In [22]:
oblig_aggreg["SZ_APPLICATION_NO"] = oblig_aggreg.index

In [23]:
oblig_aggreg.reset_index(drop=True, inplace=True)

In [24]:
oblig_aggreg = oblig_aggreg.add_prefix("oblig_")

In [25]:
df_M_2 = df_M_2.drop("_merge",axis=1)

In [26]:
df_M_3 = pd.merge(df_M_2, oblig_aggreg,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="oblig_SZ_APPLICATION_NO", indicator=True)

In [27]:
df_M_3["_merge"].value_counts()/df_M_3["_merge"].count()

left_only     0.567323
both          0.432677
right_only    0.000000
Name: _merge, dtype: float64

### Cibil score addition

In [28]:
cibil = pd.read_csv("D:/Varthana/CIBIL_DATA.csv")

In [29]:
print(cibil.shape)
print(cibil["SZ_APPLICATION_NO"].nunique())

(13623, 37)
13316


In [30]:
cibil.head()

,SZ_APPLICATION_NO,TRUST_NAME,BORROWER_CIBIL_SCORE,COBORROWER_1,COBORROWER_CIBIL_1,COBORROWER_2,COBORROWER_CIBIL_2,COBORROWER_3,COBORROWER_CIBIL_3,COBORROWER_4,...,COBORROWER_13,COBORROWER_CIBIL_13,COBORROWER_14,COBORROWER_CIBIL_14,COBORROWER_15,COBORROWER_CIBIL_15,COBORROWER_16,COBORROWER_CIBIL_16,COBORROWER_17,COBORROWER_CIBIL_17
0,S17BUB-CUT-000445,Nrusinghanath Charitable And Educational Trust,0.0,MR GOKULANANDA PANDA,675.0,MRS KALPANA MANJARI MISHRA,726.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S17BUB-JAJ-000429,Bidwan Charitable Trust,0.0,MRS PRAGYAN PARAMITA NAYAK,-1.0,MR NILAMADHAB SANGRAMSINGH,707.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S17CHA-CHA-000217,E Shiksha Asha Education Society,0.0,MR MAHENDER KUMAR,727.0,MRS POONAM DO SAJJAN KUMAR,747.0,MR VIJENDER KUMAR,526.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S17CHA-CHA-000507,Himalaya Educational Society,0.0,SARASWATI EDUCATIONAL SOCIETY,0.0,MRS SUMAN DEVI,715.0,MR SANJEEV SHARMA,-1.0,MR MAFE SINGH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,S17CHE-NGP-000407,Galaxy Educational Trust,0.0,MR RAJARAMAN G,602.0,MR KRISHNADOSS G,671.0,MRS VANI R,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
cibil_1 = pd.DataFrame(cibil[["SZ_APPLICATION_NO",
"BORROWER_CIBIL_SCORE",
"COBORROWER_CIBIL_1",
"COBORROWER_CIBIL_2",
"COBORROWER_CIBIL_3",
"COBORROWER_CIBIL_4",
"COBORROWER_CIBIL_5",
"COBORROWER_CIBIL_6",
"COBORROWER_CIBIL_7",
"COBORROWER_CIBIL_8",
"COBORROWER_CIBIL_9",
"COBORROWER_CIBIL_10",
"COBORROWER_CIBIL_11",
"COBORROWER_CIBIL_12",
"COBORROWER_CIBIL_13",
"COBORROWER_CIBIL_14",
"COBORROWER_CIBIL_15",
"COBORROWER_CIBIL_16",
"COBORROWER_CIBIL_17"]])

In [32]:
cibil_2 = cibil_1.drop_duplicates('SZ_APPLICATION_NO').sort_index()

In [33]:
df_M_3 = df_M_3.drop("_merge",axis=1)

In [34]:
cibil_2 = cibil_2.add_prefix("cibil_")

In [35]:
df_M_4 = pd.merge(df_M_3, cibil_2,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="cibil_SZ_APPLICATION_NO", indicator=True)

In [36]:
df_M_4["_merge"].value_counts()/df_M_4["_merge"].count()

both          1.0
right_only    0.0
left_only     0.0
Name: _merge, dtype: float64

### Applicant details addition


In [37]:
Applicant_det =  pd.read_csv("D:/Varthana/applicant_details_07-Sep-20.csv")

In [38]:
df_M_4 = df_M_4.drop("_merge",axis=1)

In [39]:
Applicant_det = Applicant_det.add_prefix("applDet_")

In [40]:
df_M_5 = pd.merge(df_M_4, Applicant_det,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="applDet_APPLICATION_NO", indicator=True)

In [41]:
df_M_5["_merge"].value_counts()/df_M_5["_merge"].count()

both          0.999544
left_only     0.000456
right_only    0.000000
Name: _merge, dtype: float64

### Bank Details addition

In [42]:
bank_det = pd.read_csv("D:/Varthana/BANK_DETAILS_NEW_14SEP20.csv")

In [43]:
bank_det_v2 = pd.DataFrame(bank_det[["SZ_APPLICATION_NO","TOTAL_BANKING_SUMMATION","TOTAL_LEDGER_SUMMATION","TOTAL_EXPENSES","TOTAL_OBLIGATIONS_EMI","TOTAL_ABB"]])

In [44]:
bank_det_v3 = bank_det_v2.drop_duplicates()

In [45]:
df_M_5 = df_M_5.drop("_merge",axis=1)

In [46]:
bank_det_v4 = bank_det_v3.add_prefix("bankdet_")

In [47]:
df_M_6 = pd.merge(df_M_5, bank_det_v4,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="bankdet_SZ_APPLICATION_NO", indicator=True)

In [48]:
list(df_M_6)

['LOS_Base_BRANCH_NAME',
 'LOS_Base_SPOKE_NAME',
 'LOS_Base_SOURCED_MONTH',
 'LOS_Base_SZ_APPLICATION_NO',
 'LOS_Base_TRUST_NAME',
 'LOS_Base_SCHOOL_NAME',
 'LOS_Base_SCHOOL_DISTRICT',
 'LOS_Base_PRODUCT_TYPE',
 'LOS_Base_PRODUCT_DESC',
 'LOS_Base_OPS_PERSON_NAME',
 'LOS_Base_APPL_TYPE',
 'LOS_Base_ORIGINAL_APPLICATION_NO',
 'LOS_Base_OUTSTANDING_AMT',
 'LOS_Base_LOGIN_DATE',
 'LOS_Base_BUREAU_REVIEW_START_DATE',
 'LOS_Base_REJECTION_DATE',
 'LOS_Base_LOAN_AMOUNT',
 'LOS_Base_APPROVED_LOAN_AMOUNT',
 'LOS_Base_PD_DATE',
 'LOS_Base_PD_CUSTOMER_DELAY',
 'LOS_Base_PD_DONEBY',
 'LOS_Base_LOAN_DECISION_STATUS',
 'LOS_Base_RECOMMENDED_DATE',
 'LOS_Base_SOFT_SANCTION_APPROVAL_DATE',
 'LOS_Base_LOAN_APPROVED_BY',
 'LOS_Base_LOAN_STATUS',
 'LOS_Base_FINAL_SANCTION_DATE',
 'LOS_Base_DISBURSEMENT_DATE',
 'LOS_Base_MAIN_WORKFLOW',
 'LOS_Base_LEGAL_WORKFLOW',
 'LOS_Base_DOC_COLL_DATE',
 'LOS_Base_TECHNICAL_WORKFLOW',
 'LOS_Base_FCU_WORKFLOW',
 'LOS_Base_COLLATERAL_ENTRY',
 'LOS_Base_APP_UW_COMPLETE_

In [49]:
df_M_6["_merge"].value_counts()/df_M_6["_merge"].count()

both          0.601168
left_only     0.398832
right_only    0.000000
Name: _merge, dtype: float64

#### Cashflow details addition

In [50]:
cfa_det = pd.read_csv("D:/Varthana/V_CAM_CFA_EXPENSE_07-Sep-20.csv")

In [51]:
cfa_aggreg = pd.pivot_table(cfa_det, values='EXP_AMOUNT', index=['SZ_APPLICATION_NO'],columns=['EXPENSE_TYP'], aggfunc=np.sum)

In [52]:
cfa_aggreg = cfa_aggreg.reset_index()

In [53]:
cfa_aggreg.columns = cfa_aggreg.columns.str.replace(' ', '_')
cfa_aggreg.columns = cfa_aggreg.columns.str.replace('-', '')

In [54]:
cfa_aggreg_total = pd.pivot_table(cfa_det, values='EXP_AMOUNT', index=['SZ_APPLICATION_NO'],aggfunc=np.sum)

In [55]:
cfa_aggreg_total = cfa_aggreg_total.rename(columns={ "EXP_AMOUNT": "Total_exp"})

In [56]:
cfa_aggreg_total.reset_index()

,SZ_APPLICATION_NO,Total_exp
0,K18CMB-SAL-004052,2613600.0
1,K18HUB-GUL-005114,14400.0
2,K18NAS-MAL-002987,21696000.0
3,K18THA-MAL-000815,2048400.0
4,K18THA-MAL-000816,2048400.0
...,...,...
5812,U20VIS-RJM-012576,135000.0
5813,U20VIS-RJM-013381,55000.0
5814,U20VIS-VIS-013605,200000.0
5815,U20VIS-VJN-012433,65000.0


In [57]:
cfa_aggreg_final = pd.merge(cfa_aggreg,cfa_aggreg_total,how="left",on="SZ_APPLICATION_NO")

In [58]:
df_M_6 = df_M_6.drop("_merge",axis=1)

In [59]:
cfa_aggreg_final = cfa_aggreg_final.add_prefix("cfa_")

In [60]:
df_M_7 = pd.merge(df_M_6, cfa_aggreg_final,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="cfa_SZ_APPLICATION_NO", indicator=True)

In [61]:
df_M_7["_merge"].value_counts()/df_M_7["_merge"].count()

left_only     0.539591
both          0.460409
right_only    0.000000
Name: _merge, dtype: float64

### Collateral details add

In [62]:
collat_det = pd.read_csv("D:/Varthana/COLLATERAL_DETAILS_07-Sep-20.csv")

In [63]:
collat_det_2 = pd.DataFrame(collat_det[["SZ_APPLICATION_NO","TYPE","TOTAL_AMOUNT","F_PRESENT_VALUE","F_TOTAL_VALUE","F_PER_CHARGE"]])

In [64]:
collat_det_3 = pd.pivot_table(collat_det_2, values=["TOTAL_AMOUNT","F_PRESENT_VALUE","F_TOTAL_VALUE","F_PER_CHARGE"], index=['SZ_APPLICATION_NO'], aggfunc={"TOTAL_AMOUNT":np.sum,
                                                                                                               "F_PRESENT_VALUE":np.sum,
                                                                                                               "F_TOTAL_VALUE":np.sum,
                                                                                                               "F_PER_CHARGE":np.mean})

In [65]:
collat_det_3 = collat_det_3.reset_index()

In [66]:
df_M_7 = df_M_7.drop("_merge",axis=1)

In [67]:
collat_det_4 = collat_det_3.add_prefix("collat_")

In [68]:
df_M_8 = pd.merge(df_M_7, collat_det_4,how="left",left_on="LOS_Base_SZ_APPLICATION_NO", right_on="collat_SZ_APPLICATION_NO", indicator=True)

In [69]:
df_M_8["_merge"].value_counts()/df_M_8["_merge"].count()

left_only     0.742383
both          0.257617
right_only    0.000000
Name: _merge, dtype: float64

In [70]:
df_M_8['GROSS_FEE'] = np.where(df_M_8['LOS_Base_PRODUCT_TYPE'] == 'Secured',
                        (np.where(df_M_8['bankdet_TOTAL_LEDGER_SUMMATION']>0,df_M_8['bankdet_TOTAL_LEDGER_SUMMATION'],
                                  df_M_8['schl_det_F_GROSS_REC_AMT'])),df_M_8['schl_det_F_GROSS_REC_AMT'])

### Ratios 

In [71]:
#students strength growth = Strentgh2/strength1
df_M_8['Stud_strngth_growth'] = df_M_8['schl_det_I_YEAR1_STRENGTH']/df_M_8['schl_det_I_YEAR2_STRENGTH']

# ABB per student
df_M_8['ABB_per_stud'] = df_M_8['bankdet_TOTAL_ABB']/df_M_8['schl_det_I_NET_STUDENTS']

# debt per student
df_M_8['tot_debt_per_stud'] = (df_M_8['schl_det_F_UNSECURE_LOAN'] + df_M_8['schl_det_F_SECURE_LOAN'])/df_M_8['schl_det_I_NET_STUDENTS']

# ABB to monthly income = ABB(averaged to month)/Monthly income
df_M_8['ABB_2_monthly_inc'] = df_M_8['bankdet_TOTAL_ABB']/(df_M_8['GROSS_FEE']/12)

### Avg cibil variable creation

In [72]:
## avg cibil of 5 borrowers
df_M_8['avg_cibil'] = (np.where(df_M_8['cibil_COBORROWER_CIBIL_1'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_1']>=300,df_M_8['cibil_COBORROWER_CIBIL_1'],0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_2'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_2']>=300,df_M_8['cibil_COBORROWER_CIBIL_2'],0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_3'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_3']>=300,df_M_8['cibil_COBORROWER_CIBIL_3'],0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_4'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_4']>=300,df_M_8['cibil_COBORROWER_CIBIL_4'],0),0)+np.where(df_M_8['cibil_COBORROWER_CIBIL_5'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_5']>=300,df_M_8['cibil_COBORROWER_CIBIL_5'],0),0))/(np.where(df_M_8['cibil_COBORROWER_CIBIL_1'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_1']>=300,1,0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_2'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_2']>=300,1,0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_3'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_3']>=300,1,0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_4'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_4']>=300,1,0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_5'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_5']>=300,1,0),0))


<ipython-input-72-d48982816887>:2: RuntimeWarning: invalid value encountered in true_divide
  df_M_8['avg_cibil'] = (np.where(df_M_8['cibil_COBORROWER_CIBIL_1'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_1']>=300,df_M_8['cibil_COBORROWER_CIBIL_1'],0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_2'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_2']>=300,df_M_8['cibil_COBORROWER_CIBIL_2'],0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_3'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_3']>=300,df_M_8['cibil_COBORROWER_CIBIL_3'],0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_4'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_4']>=300,df_M_8['cibil_COBORROWER_CIBIL_4'],0),0)+np.where(df_M_8['cibil_COBORROWER_CIBIL_5'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_5']>=300,df_M_8['cibil_COBORROWER_CIBIL_5'],0),0))/(np.where(df_M_8['cibil_COBORROWER_CIBIL_1'].notnull(),np.where(df_M_8['cibil_COBORROWER_CIBIL_1']>=300,1,0),0)+ np.where(df_M_8['cibil_COBORROWER_CIBIL_2'].no

In [73]:
df_M_8['Max_cibil'] = df_M_8[['cibil_COBORROWER_CIBIL_1','cibil_COBORROWER_CIBIL_2','cibil_COBORROWER_CIBIL_3','cibil_COBORROWER_CIBIL_4','cibil_COBORROWER_CIBIL_5']].max(axis=1)
df_M_8.loc[df_M_8['Max_cibil'] < 300, 'Max_cibil'] = -99999
df_M_8['Max_cibil'] = df_M_8['Max_cibil'].fillna(-99999)

In [74]:
df_M_8['cibil_COBORROWER_CIBIL_1_1'] = df_M_8['cibil_COBORROWER_CIBIL_1']
df_M_8.loc[df_M_8['cibil_COBORROWER_CIBIL_1_1'] < 300, 'cibil_COBORROWER_CIBIL_1_1'] = 99999
df_M_8['cibil_COBORROWER_CIBIL_1_1'] = df_M_8['cibil_COBORROWER_CIBIL_1_1'].fillna(99999)

df_M_8['cibil_COBORROWER_CIBIL_1_2'] = df_M_8['cibil_COBORROWER_CIBIL_2']
df_M_8.loc[df_M_8['cibil_COBORROWER_CIBIL_1_2'] < 300, 'cibil_COBORROWER_CIBIL_1_2'] = 99999
df_M_8['cibil_COBORROWER_CIBIL_1_2'] = df_M_8['cibil_COBORROWER_CIBIL_1_2'].fillna(99999)

df_M_8['cibil_COBORROWER_CIBIL_1_3'] = df_M_8['cibil_COBORROWER_CIBIL_3']
df_M_8.loc[df_M_8['cibil_COBORROWER_CIBIL_1_3'] < 300, 'cibil_COBORROWER_CIBIL_1_3'] = 99999
df_M_8['cibil_COBORROWER_CIBIL_1_3'] = df_M_8['cibil_COBORROWER_CIBIL_1_3'].fillna(99999)

df_M_8['cibil_COBORROWER_CIBIL_1_4'] = df_M_8['cibil_COBORROWER_CIBIL_4']
df_M_8.loc[df_M_8['cibil_COBORROWER_CIBIL_1_4'] < 300, 'cibil_COBORROWER_CIBIL_1_4'] = 99999
df_M_8['cibil_COBORROWER_CIBIL_1_4'] = df_M_8['cibil_COBORROWER_CIBIL_1_4'].fillna(99999)

df_M_8['cibil_COBORROWER_CIBIL_1_5'] = df_M_8['cibil_COBORROWER_CIBIL_5']
df_M_8.loc[df_M_8['cibil_COBORROWER_CIBIL_1_5'] < 300, 'cibil_COBORROWER_CIBIL_1_5'] = 99999
df_M_8['cibil_COBORROWER_CIBIL_1_5'] = df_M_8['cibil_COBORROWER_CIBIL_1_5'].fillna(99999)

In [75]:
df_M_8['Min_cibil'] = df_M_8[['cibil_COBORROWER_CIBIL_1_1','cibil_COBORROWER_CIBIL_1_2','cibil_COBORROWER_CIBIL_1_3','cibil_COBORROWER_CIBIL_1_4','cibil_COBORROWER_CIBIL_1_5']].min(axis=1)
df_M_8['Min_cibil'] = df_M_8['Min_cibil'].replace(99999,-99999)

In [76]:
df_M_8 = df_M_8.drop(['cibil_COBORROWER_CIBIL_1_1','cibil_COBORROWER_CIBIL_1_2','cibil_COBORROWER_CIBIL_1_3','cibil_COBORROWER_CIBIL_1_4','cibil_COBORROWER_CIBIL_1_5'], axis = 1) 

#### borrower cibil treatment 

In [77]:
df_M_8['cibil_BORROWER_CIBIL_SCORE'] = np.where(df_M_8['cibil_BORROWER_CIBIL_SCORE'] >= 300,
                                                 df_M_8['cibil_BORROWER_CIBIL_SCORE'],-99999)

## Final data aggregation

In [78]:
df_M_9 = df_M_8.drop(['LOS_Base_ORIGINAL_APPLICATION_NO',
                         'Loan_details_APPLICATION_NO',
                         'schl_det_SZ_APPLICATION_NO',
                         'oblig_SZ_APPLICATION_NO',
                         'cibil_SZ_APPLICATION_NO',
                         'applDet_APPLICATION_NO',
                         'bankdet_SZ_APPLICATION_NO',
                         'cfa_SZ_APPLICATION_NO',
                         'collat_SZ_APPLICATION_NO'],axis=1)

In [79]:
df_M_9.shape

(10962, 312)

In [80]:
#df_M_9.to_csv("/home/siddarth/LOS_data/All_data_without_depvar.csv")

## Summary of all variables 

In [81]:
Summ1 = pd.DataFrame(df_M_9.describe(include='all').T)

In [82]:
#Summ1.to_csv("/home/siddarth/LOS_data/All_var_desc_stats.csv")

In [83]:
df_M_9_2 = df_M_9.drop(['LOS_Base_BRANCH_NAME',
'LOS_Base_TRUST_NAME',
'LOS_Base_SCHOOL_NAME',
'LOS_Base_PRODUCT_DESC',
'schl_det_RC_VALIDITY',
'schl_det_FACILITIES_AVAILABLE',
'schl_det_SZ_SCHOOL_DESC',
'schl_det_SZREMARKS',
'applDet_PRODUCT',
'applDet_NAME_OF_APPLICANT',
'applDet_SCHOOL1_ADDRESS',
'applDet_SCHOOL1_NAME',
'applDet_SCHOOL2_ADDRESS',
'applDet_TRUST1_NAME',
'applDet_COBORROWER1_NAME',
'applDet_COBORROWER1_RESIDENCE_ADDR',
'applDet_COBORROWER2_RESIDENCE_ADDR',
'_merge',
'LOS_Base_SCHOOL_DISTRICT',
'LOS_Base_OPS_PERSON_NAME',
'LOS_Base_OUTSTANDING_AMT',
'LOS_Base_BUREAU_REVIEW_START_DATE',
'LOS_Base_REJECTION_DATE',
'LOS_Base_APPROVED_LOAN_AMOUNT',
'LOS_Base_PD_DATE',
'LOS_Base_PD_CUSTOMER_DELAY',
'LOS_Base_PD_DONEBY',
'LOS_Base_LOAN_DECISION_STATUS',
'LOS_Base_RECOMMENDED_DATE',
'LOS_Base_SOFT_SANCTION_APPROVAL_DATE',
'LOS_Base_LOAN_APPROVED_BY',
'LOS_Base_LOAN_STATUS',
'LOS_Base_FINAL_SANCTION_DATE',
'LOS_Base_MAIN_WORKFLOW',
'LOS_Base_LEGAL_WORKFLOW',
'LOS_Base_DOC_COLL_DATE',
'LOS_Base_TECHNICAL_WORKFLOW',
'LOS_Base_FCU_WORKFLOW',
'LOS_Base_COLLATERAL_ENTRY',
'LOS_Base_APP_UW_COMPLETE_DATE',
'LOS_Base_TECHNICAL_START_DATE',
'LOS_Base_LEGAL_COMPLETION_DATE',
'LOS_Base_TECHNICAL_COMPLETION_DATE',
'LOS_Base_USER_PROFILE',
'LOS_Base_USER_NAME',
'LOS_Base_ROLE_CODE',
'LOS_Base_START_DATE',
'LOS_Base_COAPPNAME',
'LOS_Base_TRANCH_CASE',
'LOS_Base_PD_SCHEDULE_PENDING',
'Loan_details_APP_LTV',
'Loan_details_CURRENT_OUTSTANDING',
'schl_det_I_NO_TEACHERS',
'schl_det_SZ_OTHER_CAT',
'schl_det_BUS_FEE_FAMOUNT',
'schl_det_CAN_FEE_FAMOUNT',
'schl_det_TUT_FEE_FAMOUNT',
'schl_det_HOST_FEE_FAMOUNT',
'schl_det_SCHOOL_BRANCHES',
'cibil_COBORROWER_CIBIL_4',
'cibil_COBORROWER_CIBIL_5',
'cibil_COBORROWER_CIBIL_6',
'cibil_COBORROWER_CIBIL_7',
'cibil_COBORROWER_CIBIL_8',
'cibil_COBORROWER_CIBIL_9',
'cibil_COBORROWER_CIBIL_10',
'cibil_COBORROWER_CIBIL_11',
'cibil_COBORROWER_CIBIL_12',
'cibil_COBORROWER_CIBIL_13',
'cibil_COBORROWER_CIBIL_14',
'cibil_COBORROWER_CIBIL_15',
'cibil_COBORROWER_CIBIL_16',
'cibil_COBORROWER_CIBIL_17',
'applDet_SCHOOL2_NAME',
'applDet_SCHOOL2_ID',
'applDet_SCHOOL2_ADDRESS',
'applDet_SCHOOL2_STATE',
'applDet_SCHOOL2_CITY',
'applDet_SCHOOL2_PIN_CODE',
'applDet_TRUST2_NAME',
'applDet_PANNO_TRUST2',
'cfa_Advertisment_expenses',
'cfa_Mgmt',
'cfa_Trustees_Salaries',
'applDet_COBORROWER2_DOB',
'LOS_Base_SPOKE_NAME',
'LOS_Base_SOURCED_MONTH',
'LOS_Base_LOGIN_DATE',
'LOS_Base_DISBURSEMENT_DATE',
'LOS_Base_RM_NAME',
'LOS_Base_RM_EMPLOYEE_NO',
'Loan_details_SZ_PRODUCT_DESC',
'Loan_details_INTEREST_RATE',
'Loan_details_TENOR',
'Loan_details_BASIC_INSTALL_AMT',
# 'Loan_details_FOIR_PERCEN',
'Loan_details_TRANCHE_AMOUNT_1',
'Loan_details_TRANCHE_AMOUNT_2',
# 'Loan_details_PURPOSE_OF_LOAN',
'schl_det_SLNO',
'schl_det_SZ_ORG_CODE',
'schl_det_I_SCHOOL_ID',
'schl_det_SCH_NAME',
'schl_det_SZ_TRUST_NAME',
'schl_det_SZ_OWNER_NAME',
'schl_det_SZ_SCHOOL_LOC',
'schl_det_I_NO_TRUST_MEMBER',
# 'schl_det_I_VINTAGE',
# 'schl_det_COLL_RECOG',
# 'schl_det_ECONOMIC_ACTIVITY',
# 'schl_det_SZ_AFFL_FROM',
# 'schl_det_SZ_AFFL_TO',
# 'schl_det_AFFLIATED_CLASSES',
'schl_det_SZ_YEAR1_PROOF',
'schl_det_SZ_YEAR2_PROOF',
'schl_det_I_APPLICANT_ID',
'schl_det_I_ITEM_KEY',
'schl_det_I_NID',
'schl_det_IYEAR',
'schl_det_CITRVERIFIED',
'schl_det_CSKIP',
'schl_det_I_LONAFRE_CO',
'schl_det_SZ_MONTHFROM',
'schl_det_DTVER_DATE',
# 'cibil_BORROWER_CIBIL_SCORE',
# 'cibil_COBORROWER_CIBIL_2',
# 'cibil_COBORROWER_CIBIL_3',
'applDet_EXISTING_CASE_ID',
'applDet_PORTFOLIO',
'applDet_NAME_OF_AUTHORISED_PERSON',
'applDet_TENOR',
'applDet_RM_NAME',
'applDet_RM_EMPID',
'applDet_CUSTOMER_NO',
'applDet_SCHOOL1_ID',
'applDet_SCHOOL1_PIN_CODE',
'applDet_PANNO_TRUST1',
'applDet_COBORROWER1_PANNO',
'applDet_COBORROWER2_NAME',
'applDet_COBORROWER2_PANNO',
'applDet_COBORROWER1_PINCODE',
'applDet_COBORROWER1_MOBILENO',
'applDet_COBORROWER1_DOB',
'applDet_COBORROWER2_PINCODE',
'applDet_COBORROWER2_MOBILENO',
'applDet_COBORROWER2_DOB'
# 'applDet_COBORROWER2_CIBILSCORE'
],axis = 1)

### remove variabels with >50% NA

In [84]:
df_M_9_3 = df_M_9_2.loc[:, df_M_9_2.isnull().mean() < 0.5]

In [85]:
list(df_M_9_2.loc[:,df_M_9_2.isnull().mean() >= 0.5].columns)

['Loan_details_FEE_COLLE_LEDGER_ELIGIBILITY',
 'Loan_details_ABB_BASED_ELIGIBILITY',
 'Loan_details_AUDITED_FINANCIAL_ELIGIBILITY',
 'Loan_details_OVERALL_LTV',
 'Loan_details_TOTAL_COLLATERAL_VALUE',
 'Loan_details_C_BUSINESS_CALL_FLAG',
 'schl_det_I_NO_STUD_USEBF',
 'schl_det_I_TRANSPORT_CHG',
 'schl_det_I_LOAN_FREE_BUSES',
 'schl_det_I_CV_LOAN_EMI',
 'schl_det_I_LONAFRE_COP',
 'oblig_CURRENT_POS',
 'oblig_EMI_AMOUNT',
 'oblig_LOAN_AMOUNT',
 'oblig_Open_emi_amt',
 'oblig_Open_loan_amt',
 'oblig_derogs',
 'oblig_pos_closed',
 'cibil_COBORROWER_CIBIL_3',
 'bankdet_TOTAL_EXPENSES',
 'bankdet_TOTAL_OBLIGATIONS_EMI',
 'cfa_Admin_Exp',
 'cfa_Misc_Expenses',
 'cfa_Nonteaching_Salaries',
 'cfa_Other_Expenses',
 'cfa_Regular_Maintenance',
 'cfa_Rent',
 'cfa_Staff_Salary',
 'cfa_Stationery',
 'cfa_Teaching_Staff_Salary',
 'cfa_Total_exp',
 'collat_F_PER_CHARGE',
 'collat_F_PRESENT_VALUE',
 'collat_F_TOTAL_VALUE',
 'collat_TOTAL_AMOUNT']

In [86]:
df_M_9_4 = df_M_9_3.drop(['applDet_SCHOOL1_STATE',
'schl_det_SAL_RANGE_TEACHER',
'schl_det_I_TOT_BOYS',
'schl_det_ACTUAL_CLASS_RUN',
'applDet_COBORROWER2_GENDER',
'schl_det_C_OWNERSHIP',
'schl_det_SZ_DIST_FROM_HUB',
'applDet_COBORROWER1_DESIGNATION',
'Loan_details_PURPOSE_OF_LOAN',
'schl_det_RK1',
'applDet_SCHOOL1_CITY',
'schl_det_TOT_SHIFTS',
'applDet_SOURCE_BRANCH',
'schl_det_I_TOT_GIRLS',
'schl_det_YEAR_OF_SCHOOL_STARTED',
'applDet_SPOKE_NAME',
'schl_det_AFFLIATED_CLASSES',
'schl_det_ECONOMIC_ACTIVITY',
'applDet_COBORROWER2_DESIGNATION',
'applDet_COBORROWER1_GENDER'],axis=1)

### formatting and imputing Loan Amount variables 

In [87]:
## replacing the commas with empty space
df_M_9_4['LOS_Base_LOAN_AMOUNT_1'] = df_M_9_4['LOS_Base_LOAN_AMOUNT']
## extracting only values before decimal
df_M_9_4['LOS_Base_LOAN_AMOUNT_2']=df_M_9_4['LOS_Base_LOAN_AMOUNT_1']
# ## converting to int
df_M_9_4['LOS_Base_LOAN_AMOUNT_3']=df_M_9_4['LOS_Base_LOAN_AMOUNT_2'].astype(float).astype(int)

In [88]:
## replacing the commas with empty space
df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_1'] = df_M_9_4['applDet_APPLIED_LOAN_AMOUNT'].str.replace(',', '')
## extracting only values before decimal
df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_2']=df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_1'].str.extract('([a-zA-Z0-9]+)',expand=True)[0]
# ## converting to int
df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_3']=df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_2'].astype(float)

#### Additional ratios

In [89]:
#Avg Loan amount requested per student  
df_M_9_4['avg_Loan_req_per_student'] = df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_3']/df_M_9_4['schl_det_I_NET_STUDENTS']

# loan amoutn to fee income ratio
df_M_9_4['ReqAmt_2_feeinc'] = df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_3']/df_M_9_4['GROSS_FEE']

### Loan to ABB
df_M_9_4['ReqAmt_2_ABB'] = df_M_9_4['applDet_APPLIED_LOAN_AMOUNT_3']/df_M_9_4['bankdet_TOTAL_ABB']

#### Fill Na and inf with special values 

In [90]:
df_M_9_4.fillna(-99999, inplace=True)
df_M_9_4.replace(np.inf, -99999, inplace=True)

## removing rupee variables 

In [91]:
df_M_9_5 = df_M_9_4.drop(['schl_det_F_RESERVES',
'schl_det_F_TRANSPORT_INCOME',
'schl_det_F_UNSECURE_LOAN',
'schl_det_REG_MAIN_EXP_FAMOUNT',
'schl_det_MGMT_EXP_FAMOUNT',
'schl_det_F_GROSS_MARGIN_VAL',
'schl_det_F_STAFF_SAL',
'schl_det_F_FEE_INCOME',
'schl_det_F_MGMT_SAL',
'Loan_details_RECOMENDED_LOAN_AMT',
'schl_det_F_INVESTMENTS',
'schl_det_MIS_EXP_FAMOUNT',
'schl_det_F_OTHER_INCOME',
'schl_det_F_TCHR_SALP',
'schl_det_NT_SAL_EXP_FAMOUNT',
'schl_det_F_OTHER_EXP',
'bankdet_TOTAL_LEDGER_SUMMATION',
'schl_det_F_OTHER_ASSETS',
'schl_det_F_PAYBLES',
'Loan_details_LTV_ELIG',
'schl_det_F_VEH_EXP',
'LOS_Base_LOAN_AMOUNT',
'LOS_Base_LOAN_AMOUNT_1',
'LOS_Base_LOAN_AMOUNT_2',
'LOS_Base_LOAN_AMOUNT_3',
'schl_det_ADMIN_EXP_FAMOUNT',
'schl_det_F_INT_EXP',
'schl_det_F_TCHR_SAL',
'schl_det_F_TOT_STAFF_SAL',
'schl_det_F_OTHER_LIA',
'schl_det_F_BLDNG_EXP',
'Loan_details_ESTIMATED_COST',
'schl_det_F_DEPRECIATION',
'schl_det_F_NONTCHR_SAL',
'schl_det_F_GRAND_TOT',
'schl_det_F_TAX_INS',
# 'schl_det_F_ANNUAL_FEE',
'Loan_details_CASHFLOW_ELIGIBILITY',
'schl_det_TEC_STF_SAL_EXP_FAMOUNT',
'schl_det_OTHER_EXP_FAMOUNT',
'schl_det_F_TOT_OTHER_INC',
'schl_det_STF_SAL_EXP_FAMOUNT',
'schl_det_F_SECURE_LOAN',
'schl_det_F_RECEIVABLES',
'schl_det_F_CAHS_BANK',
'schl_det_COAPP_INCOME',
'bankdet_TOTAL_ABB',
'schl_det_F_RENT_EXPENSES',
'bankdet_TOTAL_BANKING_SUMMATION',
'schl_det_F_SHARE_CAPITAL',
'schl_det_F_LAND_BLDNG',
'schl_det_F_GROSS_REC_AMT',
'schl_det_STATIONARY_EXP_FAMOUNT',
'schl_det_F_DONATIONS',
'schl_det_TRUST_SAL_EXP_FAMOUNT',
'schl_det_ADV_EXP_FAMOUNT',
'schl_det_F_ADMIN_EXP',
'schl_det_RENT_EXP_FAMOUNT',
'applDet_APPLIED_LOAN_AMOUNT',
'applDet_APPLIED_LOAN_AMOUNT_1',
'applDet_APPLIED_LOAN_AMOUNT_2',
'applDet_APPLIED_LOAN_AMOUNT_3'], axis = 1 )

## merging all performances to the LOS base

In [92]:
all_perf = pd.read_csv('D:/Varthana/all_perf.csv')

In [93]:
allPerf_LOS = pd.merge(df_M_9_5 , all_perf, how = 'inner', left_on = "LOS_Base_SZ_APPLICATION_NO" , right_on = "sz_loan_account_no")

In [94]:
allPerf_LOS[allPerf_LOS['ever_dpd_30_9'].notnull()].shape

(3988, 102)

In [95]:
#allPerf_LOS.to_csv("/home/siddarth/LOS_data/All_indep_all_depVar_v2.csv")

In [96]:
pd.set_option('display.max_rows', 500)
allPerf_LOS.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
LOS_Base_SZ_APPLICATION_NO,4460,4460,S18JAI-AJM-003430,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOS_Base_PRODUCT_TYPE,4460,4,Secured,2619,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOS_Base_APPL_TYPE,4460,3,Fresh,2981,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loan_details_FOIR_PERCEN,4460,NaN,NaN,NaN,-1028.68,10324.2,-99999,36.156,50.7465,58.364,314.893
Loan_details_OVERALL_LTV_PERC,4460,NaN,NaN,NaN,-35067.1,47746.3,-99999,-99999,11.8205,40.689,150
Loan_details_REC_BASIS,4460,7,CASH_FLOW,3599,NaN,NaN,NaN,NaN,NaN,NaN,NaN
schl_det_TRUST_SOCIETY,4460,7,Society,2569,NaN,NaN,NaN,NaN,NaN,NaN,NaN
schl_det_C_OWN_SCHOOL_BUILDING,4460,3,Y,3491,NaN,NaN,NaN,NaN,NaN,NaN,NaN
schl_det_SZ_DIST_FROM_SPOKE,4460,274,0,568,NaN,NaN,NaN,NaN,NaN,NaN,NaN
schl_det_SZ_LOCALITY,4460,4,SEMIU,1727,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
allPerf_LOS["LOS_Base_SZ_APPLICATION_NO"][allPerf_LOS["cibil_BORROWER_CIBIL_SCORE"]<300].count()

4431

## individual perf files creation with all indep vars 

In [98]:
X_cols = list(df_M_9_5.columns)
X_cols = set(X_cols)-{'LOS_Base_SZ_APPLICATION_NO','LOS_Base_ORIGINAL_APPLICATION_NO','Unnamed: 0'}


# X_cols = X_cols.remove("sz_loan_account_no")

In [99]:
ID_col = "LOS_Base_SZ_APPLICATION_NO"

In [100]:
def indFiles(dep_var):
    Model_data = pd.DataFrame(pd.concat([allPerf_LOS[ID_col],allPerf_LOS[X_cols],allPerf_LOS[dep_var]],axis=1))
    Model_data_2 = Model_data[Model_data[dep_var].notnull()]
#     Model_data_3 = Model_data_2.loc[:, Model_data_2.isnull().mean() < 0.5]
#     Model_data_3.fillna(-99999, inplace=True)
#     Model_data_3.replace(np.inf, -99999, inplace=True)
    return Model_data_2


In [101]:
indFiles('ever_dpd_30_9').shape

(3988, 73)

In [ ]:
indFiles('ever_dpd_30_9').to_csv('D:/Varthana/AllData_Indvar_30_9_indepvars_final_v1.csv',index=False)